In [1]:
import numpy as np

In [ ]:
def pi1 (q1,q2):
    return (np.exp(q1) /(np.exp(q1) +np.exp(q2)))

def pi2 (q1,q2):
    return (np.exp(q2) /(np.exp(q1) +np.exp(q2)))

In [ ]:
E = [s , i , r]
q1, q2 = {[0 ,1 ] , [0 ,1 ]}

for _ in q1 and _ in q2 :
        p[s][q1,q2] = 0.2

for _ in q1 and _ in q2 :
        p[i][q1,q2] = 0.05

ds_q1_q2 = 